In [ ]:
# First, install and import required packages
!pip install earthengine-api
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.6 MB/s eta 0:00:00


In [ ]:


import ee
import geemap
import folium
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='your ee login')

# Load the FAO GAUL administrative boundaries dataset
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

# Filter for Lahore in Punjab, Pakistan
lahoreRegion = admin2 \
    .filter(ee.Filter.eq('ADM2_NAME', 'Lahore District')) \
    .filter(ee.Filter.eq('ADM1_NAME', 'Punjab')) \
    .filter(ee.Filter.eq('ADM0_NAME', 'Pakistan'))

# Get the geometry
roi = lahoreRegion.geometry()

# Define scaling factors for each gas
scalingFactors = {
    'NO2': 1e5,  # Multiply by 100000
    'SO2': 1e5,  # Multiply by 100000
    'CO': 1e3,   # Multiply by 1000
    'O3': 1e2,   # Multiply by 100
    'Aerosol': 1 # No scaling needed for aerosol
}

# Define visualization parameters for each gas (with scaled values)
vizParams = {
    'NO2': {'min': 0, 'max': 20, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']},
    'SO2': {'min': 0, 'max': 50, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']},
    'CO': {'min': 0, 'max': 50, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']},
    'O3': {'min': 12, 'max': 15, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']},
    'Aerosol': {'min': -1, 'max': 2.0, 'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']}
}

def processDataset(collectionId, band, startDate, endDate, roi, gas_name):
    """Process and return an Earth Engine image collection with scaled values."""
    # Get the scaling factor for this gas
    scale_factor = scalingFactors[gas_name]

    return ee.ImageCollection(collectionId) \
        .select(band) \
        .filterDate(startDate, endDate) \
        .filterBounds(roi) \
        .mean() \
        .multiply(scale_factor) \
        .clip(roi)  # Scale up the values before clipping

# Process all datasets with scaling
datasets = {
    'NO2': processDataset('COPERNICUS/S5P/NRTI/L3_NO2', 'NO2_column_number_density',
                         '2025-01-01', '2025-01-06', roi, 'NO2'),
    'SO2': processDataset('COPERNICUS/S5P/NRTI/L3_SO2', 'SO2_column_number_density',
                         '2025-01-01', '2025-01-06', roi, 'SO2'),
    'CO': processDataset('COPERNICUS/S5P/NRTI/L3_CO', 'CO_column_number_density',
                        '2025-01-01', '2025-01-06', roi, 'CO'),
    'O3': processDataset('COPERNICUS/S5P/NRTI/L3_O3', 'O3_column_number_density',
                        '2025-01-01', '2025-01-06', roi, 'O3'),
    'Aerosol': processDataset('COPERNICUS/S5P/NRTI/L3_AER_AI', 'absorbing_aerosol_index',
                             '2025-01-01', '2025-01-06', roi, 'Aerosol')
}

# Create an interactive map
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add layers to the map with layer control
for gas, image in datasets.items():
    Map.addLayer(image, vizParams[gas], f'S5P {gas} over Lahore')

# Add Lahore boundary
style = {'color': 'red', 'fillColor': '00000000', 'width': 2}
Map.addLayer(lahoreRegion.style(**style), {}, 'Lahore Boundary')

# Display the map
display(Map)

# Function to get statistics for each gas
def get_gas_statistics(image):
    """Calculate basic statistics for a gas layer."""
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean().combine(
            ee.Reducer.minMax(), '', True),
        geometry=roi,
        scale=1000,
        maxPixels=1e9
    ).getInfo()
    return stats

# Create widgets for interactive analysis
gas_dropdown = widgets.Dropdown(
    options=list(datasets.keys()),
    description='Select Gas:',
    style={'description_width': 'initial'}
)

def on_gas_selection(change):
    """Display statistics when a gas is selected."""
    selected_gas = change['new']
    stats = get_gas_statistics(datasets[selected_gas])
    print(f"\nStatistics for {selected_gas}:")
    for key, value in stats.items():
        print(f"{key}: {value}")

gas_dropdown.observe(on_gas_selection, names='value')
display(gas_dropdown)

# Optional: Add time series analysis
def plot_time_series(gas_name, start_date, end_date):
    """Plot time series data for selected gas."""
    collection = ee.ImageCollection(f'COPERNICUS/S5P/NRTI/L3_{gas_name}') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)

    time_series = collection.map(lambda image: image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=1000,
        maxPixels=1e9
    )).aggregate_array('mean').getInfo()

    plt.figure(figsize=(12, 6))
    plt.plot(time_series)
    plt.title(f'{gas_name} Time Series')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.show()

# Example usage of time series analysis:
# plot_time_series('NO2', '2019-06-01', '2019-06-06')

Map(center=[31.466310119401168, 74.35835574868368], controls=(WidgetControl(options=['position', 'transparent_…

Dropdown(description='Select Gas:', options=('NO2', 'SO2', 'CO', 'O3', 'Aerosol'), style=DescriptionStyle(desc…


Statistics for SO2:
SO2_column_number_density_max: 115.95636606224402
SO2_column_number_density_mean: 17.962236947301342
SO2_column_number_density_min: -96.85583645484549
